In [2]:
from astroquery.vizier import Vizier

from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import ascii
from astropy.table import Table

import numpy as np
import numpy.ma as ma
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm

Define constants

In [3]:
Rg = 3.518
Rr = 2.617
Ri = 1.971
mu_true = 18.477
Fe_H = -1.48

Define functions

In [4]:
def query(catalog, vtype):
    vizier  = Vizier(columns=["[DCM2005]","RAJ2000", "DEJ2000", "VType", 
                              "Per","<Bint>", "<Vint>", "<Iint>","Notes"], 
                     column_filters={"VType":vtype})
    vizier.ROW_LIMIT = -1
    results = vizier.get_catalogs(catalog)[0]
    return results

In [5]:
def match_feh(IDlist , fehMap):
    IDarray    = np.array(IDlist)
    fehIDarray = np.array(fehMap['ID'])
    ids = np.isin(IDarray, fehIDarray)
    feh = []
    for i in range(len(ids)):
        DCM = IDlist[i]
        if ids[i] == True:
            fehid = fehMap['ID'].index(DCM)
            feh.append(fehMap['[Fe/H]'][fehid])
        else:
            feh.append(Fe_H)
    feh = np.array(feh)
    return feh

In [16]:
def get_LMCmu(lp, mag, feh, zp, slope, met, title_in):

    M = zp + slope*lp + met*feh
    delta_mu = (mag - M) - mu_true
    N = len(lp)

    dmean = np.mean(delta_mu)
    dmedian = np.median(delta_mu)
    print("%s\t: %2.3f\t%2.3f" % (title_in, dmean, dmedian))
    
    # plt.minorticks_on()
    # plt.tick_params(which='both', bottom='on', top='on', left='on', right='on', direction='inout')
    # plt.hist(delta_mu, bins=50, color='skyblue', edgecolor='black')
    # plt.hist(delta_mu, bins=int(N/10)+8, color='skyblue', edgecolor='black')
    # plt.axvline(0.0, c='r', ls='-')
    # plt.axvline(dmean, c='r', ls='--')
    # plt.ylabel("Count", fontsize=13)
    # plt.xlabel(r"$\mu_j-\mu_{LMC}^0$", fontsize=15)
    # plt.title(str(title_in))
    # plt.tight_layout()
    # plt.show()

Read reddening map

In [7]:
reddening_map = pd.read_csv(
                            "EBV_res3_lmc.txt", 
                            sep=r"\s+", 
                            comment='#',  # Ignore comment lines
                            names=['RA', 'Dec', 'E_BV', 'Error', 'Sigma_RC', 'N_RC']
                           )
map_coords = SkyCoord(ra=reddening_map["RA"].values * u.deg, dec=reddening_map['Dec'].values * u.deg)

Read [Fe/H] map

In [8]:
feh_map = pd.read_csv(
                      "metallicity.txt", 
                      sep=r"\s+", 
                      comment='#',  # Ignore comment lines
                      names=['ID', 'Field', 'B-V', 'P', '[Fe/H]', 'E(B-V)']
                     )
feh_map = feh_map.to_dict('list')

Query

In [9]:
type_ab = query("J/A+A/430/603/variables", "ab")
type_c  = query("J/A+A/430/603/variables", "c")

print("All type ab:", len(type_ab))
print("All type c:", len(type_c))

# Remove "Incomplete" and "Blend"
valid_ab = (type_ab['Notes'] != "Incomplete") & (type_ab['Notes'] != "Blend")
valid_c  = (type_c['Notes'] != "Incomplete") & (type_c['Notes'] != "Blend")

rrab = type_ab[valid_ab]
rrc  = type_c[valid_c]

print("Complete type ab:", len(rrab))
print("Complete type c:", len(rrc))

All type ab: 101
All type c: 54
Complete type ab: 94
Complete type c: 54


# RRab

In [21]:
ID_ab = rrab['__DCM2005_']
P_ab = np.log10(rrab['Per'])

# Create SkyCoord objects in a vectorized way
Coord_ab = SkyCoord(
    ra=rrab['RAJ2000'],
    dec=rrab['DEJ2000'],
    unit=(u.hourangle, u.deg),
    frame='icrs'
)

BVab = rrab['__Bint_'] - rrab['__Vint_']

g_ab = rrab['__Bint_'] - 0.108 - 0.485*BVab - 0.032*BVab*BVab
r_ab = rrab['__Vint_'] + 0.082 - 0.462*BVab + 0.041*BVab*BVab
i_ab = rrab['__Iint_'] + 0.341 + 0.154*BVab - 0.025*BVab*BVab

E_BV_array = np.array([reddening_map['E_BV'].iloc[coord.match_to_catalog_sky(map_coords)[0]] for coord in Coord_ab])

g0_ab = g_ab# - Rg*E_BV_array
r0_ab = r_ab# - Rr*E_BV_array
i0_ab = i_ab# - Ri*E_BV_array

flag_gab = g0_ab < 99
flag_rab = r0_ab < 99
flag_iab = i0_ab < 99

FeH_ab = match_feh(ID_ab, feh_map)

In [23]:
get_LMCmu(P_ab[flag_gab], g0_ab[flag_gab], FeH_ab[flag_gab], 0.649, -0.302, 0.159, "Ngeow g-band ab-PLZ")
get_LMCmu(P_ab[flag_rab], r0_ab[flag_rab], FeH_ab[flag_rab], 0.337, -1.090, 0.139, "Ngeow r-band ab-PLZ")
get_LMCmu(P_ab[flag_iab], i0_ab[flag_iab], FeH_ab[flag_iab], 0.243, -1.432, 0.144, "Ngeow i-band ab-PLZ")

get_LMCmu(P_ab[flag_gab]-P0, g0_ab[flag_gab], FeH_ab[flag_gab]-Z0, 0.794, -0.527, 0.264, "Narloch g-band ab-PLZ(1)")
get_LMCmu(P_ab[flag_rab]-P0, r0_ab[flag_rab], FeH_ab[flag_rab]-Z0, 0.651, -1.230, 0.205, "Narloch r-band ab-PLZ(1)")
get_LMCmu(P_ab[flag_iab]-P0, i0_ab[flag_iab], FeH_ab[flag_iab]-Z0, 0.635, -1.682, 0.174, "Narloch i-band ab-PLZ(1)")

Ngeow g-band ab-PLZ	: 0.525	0.519
Ngeow r-band ab-PLZ	: 0.421	0.427
Ngeow i-band ab-PLZ	: 0.365	0.359
Narloch g-band ab-PLZ(1)	: 0.221	0.206
Narloch r-band ab-PLZ(1)	: 0.174	0.168
Narloch i-band ab-PLZ(1)	: 0.119	0.119


In [22]:
indat = ascii.read("DiF_in.csv", format='csv')

mi = indat['Imag'] < 99.999
mab = indat['rtype'] == 'ab'
mc = indat['rtype'] == 'c'

logP = np.log10(indat['period'])
FeH = indat['FeH']

BV = indat['Bmag'] - indat['Vmag']

gmag = indat['Bmag'] - 0.108 - 0.485*BV - 0.032*BV*BV
rmag = indat['Vmag'] + 0.082 - 0.462*BV + 0.041*BV*BV
imag = indat['Imag'] + 0.341 + 0.154*BV - 0.025*BV*BV

g0 = gmag#- Rg*indat['EBV']
r0 = rmag#- Rr*indat['EBV']
i0 = imag#- Ri*indat['EBV']

ilogP = logP[mi]
iFeH = FeH[mi]
ii0 = i0[mi]
iab = mab[mi]
ic = mc[mi]

# my PLZ
get_LMCmu(logP[mab], g0[mab], FeH[mab], 0.649, -0.302, 0.159, "Ngeow g-band ab-PLZ")
get_LMCmu(logP[mab], r0[mab], FeH[mab], 0.337, -1.090, 0.139, "Ngeow r-band ab-PLZ")
get_LMCmu(ilogP[iab], ii0[iab], iFeH[iab], 0.243, -1.432, 0.144, "Ngeow i-band ab-PLZ")

# Narloch PLZ
P0 = -0.25
Z0 = -1.5

get_LMCmu(logP[mab]-P0, g0[mab], FeH[mab]-Z0, 0.794, -0.527, 0.264, "Narloch g-band ab-PLZ(1)")
get_LMCmu(logP[mab]-P0, r0[mab], FeH[mab]-Z0, 0.651, -1.230, 0.205, "Narloch r-band ab-PLZ(1)")
get_LMCmu(ilogP[iab]-P0, ii0[iab], iFeH[iab]-Z0, 0.635, -1.682, 0.174, "Narloch i-band ab-PLZ(1)")

Ngeow g-band ab-PLZ	: 0.525	0.519
Ngeow r-band ab-PLZ	: 0.421	0.427
Ngeow i-band ab-PLZ	: 0.365	0.359
Narloch g-band ab-PLZ(1)	: 0.221	0.206
Narloch r-band ab-PLZ(1)	: 0.174	0.168
Narloch i-band ab-PLZ(1)	: 0.119	0.119


## RRc

In [20]:
ID_c = rrc['__DCM2005_']
P_c = np.log10(rrc['Per'])

# Create SkyCoord objects in a vectorized way
Coord_c = SkyCoord(
    ra=rrc['RAJ2000'],
    dec=rrc['DEJ2000'],
    unit=(u.hourangle, u.deg),
    frame='icrs'
)

BVc  = rrc['__Bint_'] - rrc['__Vint_']
g_c  = rrc['__Bint_'] - 0.108 - 0.485*BVc - 0.032*BVc*BVc
r_c  = rrc['__Vint_'] + 0.082 - 0.462*BVc + 0.041*BVc*BVc
i_c  = rrc['__Iint_'] + 0.341 + 0.154*BVc - 0.025*BVc*BVc

E_BV_array = np.array([reddening_map['E_BV'].iloc[coord.match_to_catalog_sky(map_coords)[0]] for coord in Coord_c])

g0_c = g_c- Rg*E_BV_array
r0_c = r_c- Rr*E_BV_array
i0_c = i_c- Ri*E_BV_array

flag_gc = g0_c < 99
flag_rc = r0_c < 99
flag_ic = i0_c < 99

FeH_c = match_feh(ID_c, feh_map)